In [8]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# Question 1

In [9]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [10]:
ipo_filings = ipo_dfs[0]
ipo_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     324 non-null    object
 1   Symbol          324 non-null    object
 2   Company Name    324 non-null    object
 3   Price Range     324 non-null    object
 4   Shares Offered  324 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [11]:
# Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).

ipo_filings['Filing Date'] = pd.to_datetime(ipo_filings['Filing Date'], format='%b %d, %Y')
ipo_filings['Shares Offered'] = pd.to_numeric(ipo_filings['Shares Offered'], errors='coerce')
ipo_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     324 non-null    datetime64[ns]
 1   Symbol          324 non-null    object        
 2   Company Name    324 non-null    object        
 3   Price Range     324 non-null    object        
 4   Shares Offered  250 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 12.8+ KB


In [12]:
ipo_filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN


In [13]:
ipo_filings['Price Range']

0                   -
1               $3.00
2              $10.00
3              $10.00
4                   -
            ...      
319    $8.00 - $10.00
320            $10.00
321    $8.00 - $10.00
322             $4.00
323     $5.00 - $6.50
Name: Price Range, Length: 324, dtype: object

In [14]:
def get_avg_price(price_range):
    if price_range == '-':
        return np.nan
    prices = price_range.split(' - ')
    if len(prices) == 1:
        return float(prices[0].strip('$'))
    else:
        price1, price2 = prices
        return (float(price1.strip('$')) + float(price2.strip('$'))) / 2

ipo_filings['Avg_price'] = ipo_filings['Price Range'].apply(get_avg_price)

In [15]:
ipo_filings['Shares_offered_value'] = ipo_filings['Shares Offered'] * ipo_filings['Avg_price']

In [16]:
filter = (ipo_filings['Filing Date'].dt.year == 2023) & (ipo_filings['Filing Date'].dt.dayofweek == 4)
filtered_ipo_filings = ipo_filings[filter]

In [17]:
total_shares_offered = round(filtered_ipo_filings['Shares_offered_value'].sum(skipna=True) / 1000000)

In [18]:
total_shares_offered

286

# Question 2

In [19]:
url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [20]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [21]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [22]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      67 non-null     object
 1   Symbol        67 non-null     object
 2   Company Name  67 non-null     object
 3   IPO Price     67 non-null     object
 4   Current       67 non-null     object
 5   Return        67 non-null     object
dtypes: object(6)
memory usage: 3.3+ KB


In [23]:
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [24]:
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')

In [25]:
# "IPO date" before March 1, 2024 ("< 2024-03-01")
filter = stacked_ipos_df['IPO Date'] < '2024-03-01'
stacked_ipos_df = stacked_ipos_df[filter]

In [26]:
# revise ticker list
tickers = list(stacked_ipos_df.Symbol)
tickers.remove('RYZB')
tickers.remove('PTHR')
tickers.append('HOVR')

In [27]:
# Initialize a DataFrame to store the min_dates and growth data for all tickers
all_data_df = pd.DataFrame()

for ticker in tickers:
    print(ticker)
    ticker_df = yf.download(tickers=ticker, period='max', interval='1d')

    # Store the min_date for the current ticker
    min_date = ticker_df.index.min()

    # Generate additional columns for growth_future_1d to growth_future_30d
    for days in range(1, 31):
        ticker_df[f'growth_future_{days}d'] = ticker_df['Adj Close'].shift(-days) / ticker_df['Adj Close']

    # Add the ticker and min_date columns to the ticker_df
    ticker_df['Ticker'] = ticker
    ticker_df['Min_Date'] = min_date

    # Append the ticker_df to the all_data_df
    all_data_df = pd.concat([all_data_df, ticker_df])



SMXT


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


VHAI
DYCQ


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


CHRO
UMAC


...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


SKWD
ISRL


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


MGOL
HOVR


[*********************100%%**********************]  1 of 1 completed


In [28]:
# Filter the data to include only the rows on or after the min_date for each ticker
all_data_df = all_data_df[all_data_df.index >= all_data_df['Min_Date']]

growth_percentiles = all_data_df[[f'growth_future_{days}d' for days in range(1, 31)]].describe()



In [29]:
growth_percentiles.tail()

,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
min,0.028674,0.003584,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,...,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729
25%,0.977654,0.966667,0.957729,0.949803,0.942137,0.934689,0.927924,0.921222,0.914713,0.907682,...,0.840909,0.835821,0.831029,0.824304,0.820692,0.814348,0.811013,0.806838,0.802768,0.798491
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.996791,0.996209,0.996045,0.995598,0.995539,0.994633,0.994231,0.994077,0.993301,0.993171
75%,1.014885,1.020202,1.025230,1.028352,1.031822,1.034062,1.035883,1.038922,1.040401,1.042626,...,1.056492,1.057589,1.057954,1.059263,1.059301,1.058974,1.061404,1.063092,1.064870,1.065786
max,6.826923,6.971311,7.625593,8.650538,8.952632,10.309092,10.183544,10.380646,9.464706,9.105319,...,12.634409,11.491442,12.486842,12.284789,12.661000,15.025219,14.782093,19.655310,22.918785,27.198465


In [30]:
growth_percentiles.loc['75%'].head()

growth_future_1d    1.014885
growth_future_2d    1.020202
growth_future_3d    1.025230
growth_future_4d    1.028352
growth_future_5d    1.031822
Name: 75%, dtype: float64

In [31]:
# Find the optimal number of days X that maximizes the 75th percentile growth
max_75th_percentile = growth_percentiles.loc['75%'].max()
optimal_days = growth_percentiles.loc['75%'].idxmax().split('_')[2][:-1]

# Print the results
print("Optimal number of days:", optimal_days)
print("Maximum 75th percentile growth:", max_75th_percentile)

Optimal number of days: 30
Maximum 75th percentile growth: 1.0657859850451863


# Question 3

In [32]:
# get the data of LARGEST_STOCKS
# growth_7d
# 2014-01-01 to 2023-12-31
# 7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']
LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

start_date = '2013-12-01'  # Set start date a month earlier
end_date = '2023-12-31'

data = pd.DataFrame({'A' : []})
for ticker in LARGEST_STOCKS:

  stock_data = yf.download(ticker, start=start_date, end=end_date)
  stock_data['Ticker'] = ticker


  if data.empty:
    data = stock_data
  else:
    data = pd.concat([data, stock_data])



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [33]:
data['growth_7d'] = data.groupby('Ticker')['Close'].pct_change(7)

In [34]:
data.sample(5)

,Open,High,Low,Close,Adj Close,Volume,Ticker,growth_7d
Date,,,,,,,,
2015-03-12,144.339996,145.729996,144.009995,145.669998,145.669998,3539600,BRK-B,-0.001782
2015-08-03,99.339996,99.379997,97.309998,98.139999,89.927818,817900,ASML,-0.037182
2014-02-13,1053.500000,1069.199951,1048.250000,1066.875000,866.542297,2094350,TCS.NS,-0.008181
2014-09-08,65.980003,66.059998,65.239998,65.400002,36.668453,707300,TTE,-0.005021
2023-09-25,2350.399902,2360.699951,2335.100098,2340.449951,2340.449951,7127442,RELIANCE.NS,-0.045124


In [35]:
# Filter data from 2014-01-01 to 2023-12-31
filtered_data = data.loc['2014-01-01':'2023-12-31']

# Group by date and calculate the mean of growth_7d across all tickers for each date
average_daily_growth_7d_LARGEST = filtered_data.groupby(filtered_data.index)['growth_7d'].mean()

In [36]:
# get the list of LARGE_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']
NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']
NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']
LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [37]:
data_LARGE = pd.DataFrame({'A' : []})
for ticker in LARGE_STOCKS:

  stock_data = yf.download(ticker, start=start_date, end=end_date)
  stock_data['Ticker'] = ticker


  if data_LARGE.empty:
    data_LARGE = stock_data
  else:
    data_LARGE = pd.concat([data_LARGE, stock_data])

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [38]:
data_LARGE['growth_7d'] = data_LARGE.groupby('Ticker')['Close'].pct_change(7)

In [39]:
# Filter data from 2014-01-01 to 2023-12-31
filtered_data_LARGE = data_LARGE.loc['2014-01-01':'2023-12-31']

# Group by date and calculate the mean of growth_7d across all tickers for each date
average_daily_growth_7d_LARGE = filtered_data_LARGE.groupby(filtered_data_LARGE.index)['growth_7d'].mean()

In [40]:
# calculate the percentage of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP
round(sum((average_daily_growth_7d_LARGE - average_daily_growth_7d_LARGEST)>0)/len(average_daily_growth_7d_LARGE) * 100)

47

# QUESTION 4

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
# total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)
CCI_data = pd.read_parquet('/content/drive/MyDrive/colab data/stocks_df_combined_trunc_2014_2023.parquet.brotli')

In [134]:
subset_cci = CCI_data[['Adj Close_x', 'Month', 'Weekday', 'Ticker', 'cci']]

In [135]:
subset_cci.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80762 entries, 7011 to 5342
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Adj Close_x  80762 non-null  float64       
 1   Month        80762 non-null  datetime64[ns]
 2   Weekday      80762 non-null  int32         
 3   Ticker       80762 non-null  object        
 4   cci          80749 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(1)
memory usage: 5.4+ MB


In [136]:
subset_cci.head()

,Adj Close_x,Month,Weekday,Ticker,cci
7011,31.233059,2014-01-01,3,MSFT,57.700615
7012,31.022930,2014-01-01,4,MSFT,1.373763
7013,30.367352,2014-01-01,0,MSFT,-96.631259
7014,30.602673,2014-01-01,1,MSFT,-83.904297
7015,30.056356,2014-01-01,2,MSFT,-147.855135


In [137]:
subset_cci['sell_price'] = subset_cci.groupby('Ticker')['Adj Close_x'].shift(-5)


<ipython-input-137-be4bc8620de0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_cci['sell_price'] = subset_cci.groupby('Ticker')['Adj Close_x'].shift(-5)


In [139]:
buy_data = subset_cci[(subset_cci['cci'] > 200) & (subset_cci['Weekday'] == 4)]
buy_data['stocks_bought'] = 1000 / buy_data['Adj Close_x']
buy_data['total_sell'] = buy_data['stocks_bought'] * buy_data['sell_price']
buy_data['gross_profit'] = buy_data['total_sell'] - 1000
total_gross_profit = buy_data['gross_profit'].sum()

<ipython-input-139-242c23f27071>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_data['stocks_bought'] = 1000 / buy_data['Adj Close_x']
<ipython-input-139-242c23f27071>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_data['total_sell'] = buy_data['stocks_bought'] * buy_data['sell_price']
<ipython-input-139-242c23f27071>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [140]:
result = round(total_gross_profit / 1000)

In [142]:
buy_data.head()

,Adj Close_x,Month,Weekday,Ticker,cci,sell_price,stocks_bought,total_sell,gross_profit
7113,34.912762,2014-05-01,4,MSFT,203.518798,35.373264,28.642821,1013.190094,13.190094
7181,39.395618,2014-09-01,4,MSFT,257.013330,40.073509,25.383533,1017.207263,17.207263
7340,41.630741,2015-04-01,4,MSFT,340.009871,42.317780,24.020711,1016.503151,16.503151
7452,40.151123,2015-10-01,4,MSFT,206.125828,41.507988,24.905904,1033.793947,33.793947
7467,46.583046,2015-10-01,4,MSFT,431.107024,46.380405,21.467038,995.649908,-4.350092


In [143]:
total_gross_profit

1069.5071529360362

In [141]:
result

1

In [ ]:
# gonna loss money if tranaction fees considered. Transaction fees estimated for a series of actions is ~ $2000